In [2]:
# Libraries import
import math as m
import pygame
from pygame.draw import *
from pygame.mixer import *
from random import *
import time
pygame.init()

# --Params--

# -Auxilary-
object_list = []
clock = pygame.time.Clock()
finished = False
rocket_list = []
start_time = time.time()

# -Graphics-

# Graphiс data
general_color_list =  [(0, 0, 0), # Black
                        (127, 255, 212), # Aquamarine
                        (192, 192, 192), # Silver
                        (255, 0, 0), # Red
                        (0, 0, 255), # Blue
                        (0, 255, 255), # Cyan
                        (255, 140, 0), # Dark orange
                        (30, 144, 255), # Fake blue
                        (0, 128, 0), # Green
                        (124, 252, 0), # Green grass
                        (40, 200, 170), # Green spring
                        (240, 230, 140), # Haki
                        (219, 112, 147), # Heavy pink
                        (75, 0, 130), # Indigo
                        (244, 255, 255), # Light blue
                        (245, 245, 220), # Light white
                        (255, 0, 255), # Magenta
                        (25, 25, 112), # Midnight blue
                        (128, 0, 128), # Purple
                        (255, 20, 100), # Pink
                        (255, 255, 0), # Yellow
                       ]

# Screen
FPS = 24
screen_width = 1500
screen_height = 800
screen = pygame.display.set_mode((screen_width, screen_height), pygame.FULLSCREEN)

# -Objects-

# Balls
max_ball_radius = 50
max_ball_speed = 10
min_ball_radius = 25
min_ball_speed = 5
objects_amount = 5

# Rockets
max_rocket_height = 200
max_rocket_speed = 45
min_rocket_height = 100
min_rocket_speed = 20
rocket_generation_chance = 20
rocket_time_hold = 1 # Time without rocket's generation
rocket_y_x_ratio = 329 / 51 # FIXME This number was taken from a paticular image
score_per_rocket = 10

# -Sound-

# Effects
ball_delete_sound = Sound('ball_delete_sound.wav')
rocket_delete_sound = Sound('rocket_delete_sound.wav')
rocket_launch_sound = Sound('rocket_launch_sound.wav')

# Music
music.load('background_music.wav')

# -User interface-
game_time = 30
leader_amount = 5
score = 0

def make_reflection(ball: list):
    '''
    Returns an angle in rads between x axis and ball's speed direction after the reflection
    ball is list of ball params
    '''
    
    radius = ball[0]
    direction = ball[1]
    x = ball[3]
    y = ball[4]
    if x < radius:
        direction = random() * m.pi - m.pi / 2
    elif x > screen_width - radius:
        direction = random() * m.pi + m.pi / 2
    elif y < radius:
        direction = random() * m.pi - m.pi
    elif y > screen_height - radius:
        direction = random() * m.pi
    return direction
    
    
def count_rocket_score(rocket: list, rocket_list: list):
    '''
    Checks if click has occured inside the rocket
    rocket is list with rocket params
    rocket_list is list with all rockets in the game
    '''
    
    click_x = event.pos[0]
    click_y = event.pos[1]
    height = rocket[0]
    x = rocket[2]
    y = rocket[3]
    width = height / rocket_y_x_ratio
    if click_x >= x and click_x <= x + width and click_y >= y and click_y <= y + height:
        delta_score = score_per_rocket
        rocket_list.remove(rocket)
        rocket_delete_sound.play()
    else:
        delta_score = 0
    new_score = score + delta_score
    return new_score
    
    
def count_score(ball: list, object_list: list, rocket_list: list):
    '''
    Checks if click has occured inside the circle
    ball is list with ball params
    object_list is list of all balls in game
    rocket_list is list of all rockets in game
    '''
    
    click_x = event.pos[0]
    click_y = event.pos[1]
    radius = ball[0]
    x = ball[3]
    y = ball[4]
    distance = get_distance(x, y, click_x, click_y)
    if distance <= radius:
        delta_score = 1 # Increace score by one
        object_list.remove(ball)
        ball_delete_sound.play()
        generate_object(1, object_list, rocket_list) # Generate 1 ball
    else:
        delta_score = 0 # Don't increace score
    new_score = score + delta_score
    return new_score


def check_time():
    '''
    Checks if the game time is over
    '''
    
    
    if time.time() - start_time > game_time:
        return True
    else:
        return False
    
    
def get_distance(x_1: int, y_1: int, x_2: int, y_2: int):
    '''
    Gets distance between 2 points
    x_1 is x coordinate of 1 point
    y_1 is y coordinate of 1 point
    x_2 is x coordinate of 2 point
    y_2 is y coordinate of 2 point
    '''
    
    distance = m.sqrt((x_1-x_2) ** 2 + (y_1-y_2) ** 2)
    return distance
    
def generate_rocket(rocket_list: list):
    '''
    Generates a rocket on screen
    rocket_list is list of all rockets
    '''
    
    height = randint(min_rocket_height, max_rocket_height)
    speed = randint(min_rocket_speed, max_rocket_speed)
    width = int(height / rocket_y_x_ratio)
    x = randint(0, screen_width - width) # Random x coordinate to draw all image on screen
    y = screen_height
    rocket_list.append([height, speed, x, y])
    rocket_launch_sound.play()
    
    
def generate_object(objects_amount: int, object_list: list, rocket_list: list):
    '''
    Generates objects on the screen
    objects_amount is amount of objects to generate
    object_list is list of all balls in game
    rocket_list is list of all rockets in game
    '''
    
    if rocket_needed():
        generate_rocket(rocket_list)
    for ball in range(objects_amount):
        #color = general_color_list[randint(1, len(general_color_list) - 1)] # Ball color from list but not black
        radius = randint(min_ball_radius, max_ball_radius)
        speed_direction = 2 * m.pi * random() # Speed direction from 0 to 360 degrees
        speed = randint(min_ball_speed, max_ball_speed)
        x = randint(radius, screen_width - radius)
        y = randint(radius, screen_height - radius)
        object_list.append([radius, speed_direction, speed, x, y])
    
    
# def new_ball(ball: list):
#     '''
#     Draws new ball on the screen
#     '''
    
#     color = ball[0]
#     radius = ball[1]
#     x = ball[4]
#     y = ball[5]
#     circle(screen, color, (x, y), radius)
    
    
def new_satellite(height: int, x: int, y: int):
    '''
    Draws satellite on the screen
    height is height of satellite in pixels
    zero-point is upper-left point of rect inside witch rocket is drawn
    x is x coordinate of zero-point
    y is y coordinate of zero-point
    '''
    
    satellite_surface = pygame.image.load('glonass.bmp')
    scale = satellite_surface.get_height() / height
    width = int(satellite_surface.get_width() / scale)
    satellite_surface = pygame.transform.scale(satellite_surface, (width, height))
    satellite_rect = satellite_surface.get_rect(topleft = (x, y))
    screen.blit(satellite_surface, satellite_rect)

    
def new_rocket(height: int, x: int, y: int):
    '''
    Draws rocket on the screen
    height is height of rocket in pixels
    zero-point is upper-left point of rect inside witch rocket is drawn
    x is x coordinate of zero-point
    y is y coordinate of zero-point
    '''
    
    rocket_surface = pygame.image.load('soyuz-rocket-2.bmp')
    scale = rocket_surface.get_height() / height
    width = int(rocket_surface.get_width() / scale)
    rocket_surface = pygame.transform.scale(rocket_surface, (width, height))
    rocket_rect = rocket_surface.get_rect(topleft = (x, y))
    screen.blit(rocket_surface, rocket_rect)


# def move_ball(ball: list):
#     '''
#     Moves ball with wall reflections
#     ball is list with ball params
#     '''
    
#     color = ball[0]
#     direction = ball[2]
#     radius = ball[1]
#     speed = ball[3]
#     x = ball[4]
#     y = ball[5]
#     direction = make_reflection(ball)
#     x = int(x + speed * m.cos(direction))
#     y = int(y - speed * m.sin(direction))
#     new_ball(ball)
#     ball[2] = direction
#     ball[4] = x
#     ball[5] = y
   

    
def move_satellite(ball: list):
    '''
    Moves ball with wall reflections
    ball is list with ball params
    '''
    
    direction = ball[1]
    height = ball[0]
    speed = ball[2]
    x = ball[3]
    y = ball[4]
    direction = make_reflection(ball)
    x = int(x + speed * m.cos(direction))
    y = int(y - speed * m.sin(direction))
    new_satellite(height, x, y)
    ball[1] = direction
    ball[3] = x
    ball[4] = y

    
def move_rocket(rocket: list, rocket_list: list):
    '''
    Flies rocket over the screen
    rocket is list with rocket params
    rocket_list is list of all rockets in game
    '''
    
    height = rocket[0]
    speed = rocket[1]
    x = rocket[2]
    y = rocket[3]
    width = height / rocket_y_x_ratio
    y = int(y - speed)
    if y < - height:
        rocket_list.remove(rocket)
    new_rocket(height, x, y)
    rocket[3] = y
    
def rocket_needed():
    '''
    Returns 'True' if rocket is needed to generate
    '''
    
    rocket_generation = randint(1, 100) # Random chance generation
    if rocket_generation <= rocket_generation_chance and time.time() - start_time > rocket_time_hold:
        return True
    else:
        return False

def set_leaderboard(score: int):
    '''
    Sves player result to the leaderboard
    score is score amount of player
    '''
    
    leader_list = []
    output = open('leaderboard.txt', 'r')
    for line in output:
        line = line.strip()
        leader_list.append(line)
    output.close()
    leader_list = sort_leaderboard(leader_list)
    output = open('leaderboard.txt', 'w')
    for leader in leader_list:
        print(leader, file = output)
    output.close()
    

def sort_leaderboard(leader_list: list):
    '''
    Sorts leaderboard
    leader_list is list to sort
    '''
    leader_data = []
    leader_score = []
    current_time = time.asctime()
    leader_list.append('Time -> '+str(current_time)+'; score -> '+str(score))
    for leader in leader_list:
        leader_score.append(int(leader[43:])) # Part of string with score as int number
        leader_data.append(leader[:43]) # Another part of string
    for i in range(len(leader_score) - 1): # Sort
        for j in range(i+1, len(leader_score)):
            if leader_score[j] > leader_score[i]:
                swap = leader_score[i]
                leader_score[i] = leader_score[j]
                leader_score[j] = swap
                swap = leader_data[i]
                leader_data[i] = leader_data[j]
                leader_data[j] = swap
    leader_list = []
    for i in range(len(leader_data)):
        leader_list.append(leader_data[i]+str(leader_score[i]))
    leader_list = leader_list[0:leader_amount]
    return leader_list
    

music.play()
generate_object(objects_amount, object_list, rocket_list)
while not finished:
    clock.tick(FPS)
    for ball in object_list:
        move_satellite(ball)
    for rocket in rocket_list:
        move_rocket(rocket, rocket_list)
    pygame.display.update()
    screen.fill(general_color_list[0])
    finished = check_time()
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            finished = True
        elif event.type == pygame.MOUSEBUTTONDOWN:
            for ball in object_list:
                score = count_score(ball, object_list, rocket_list)
            for rocket in rocket_list:
                score = count_rocket_score(rocket, rocket_list)
pygame.quit()
set_leaderboard(score)
print(score)

22
